In [ ]:
!pip install geemap --upgrade
!pip install earthengine-api --upgrade

In [ ]:
import ee

In [ ]:
import geemap

In [ ]:
Map = geemap.Map()
Map

In [ ]:
map_mrmra = geemap.Map()
point = ee.Geometry.Point([27.8624, 40.4546])

image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
    .filterBounds(point)  \
    .filterDate("2018-01-01", "2019-12-31") \
    .sort("CLOUD_COVER") \
    .first()  \
    .select("B[1-7]")

vis_parameters = {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"]}

map_mrmra.centerObject(point, 8)
map_mrmra.addLayer(image, vis_parameters, "Landsat-8")
map_mrmra

In [ ]:
ee.Algorithms.If(ee.List(image.propertyNames()).contains("system:time_start"), True, False).getInfo()

In [ ]:
props = geemap.image_props(image)
props.getInfo()

In [ ]:
props.get("IMAGE_DATE").getInfo()

In [ ]:
props.get("CLOUD_COVER").getInfo()

In [ ]:
#Map_1.user_roi.getInfo()

In [ ]:
#region = Map_1.user_roi

In [ ]:
training = image.sample(**{
#    "region": region,
    "scale": 30,
    "numPixels": 5000,
    "seed": 0,
    "geometries": True
})
map_mrmra.addLayer(training, {}, "training", False)
map_mrmra

In [ ]:
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
result= image.cluster(clusterer)

map_mrmra.addLayer(result.randomVisualizer(), {}, "clusters")
map_mrmra

In [ ]:
legend_keys = ["Water", "Agricultural", "Forest", "Agriculture"]
legend_colors = ["#3CC7FF", "#FF4848", "#009612", "#FFFFB3"]
result = result.remap([0, 1, 2, 3], [1, 2, 3, 4])

map_mrmra.addLayer(result, {"min": 1, "max": 4, "palette": legend_colors}, "Labelled Clusters")
map_mrmra.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, poisiton="bottomright")
map_mrmra

In [ ]:
print("Change Layer Opacity:")
cluster_layer = map_mrmra.layers[-1]
cluster_layer.interact(opacity = (0, 1, 0.1))

In [ ]:
import os
out_dir = os.path.join(os.path.expanduser("∼"), "Downloads")
out_file = os.path.join(out_dir, "cluster.tif")

In [ ]:
geemap.ee_export_image(result, filename=out_file, scale=90)